In [1]:
# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

## VaR con Títulos

In [2]:
def  VaRPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                    start: 'Fecha inicio',
                    end: 'Fecha final',
                    titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                    confianza: 'Nivel de confianza del VaR (en decimal)'
                    ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Acomodarlo menor a mayor
    sortport= np.sort(Port)
    n= len(Port)
    # Nivel de confianza
    fuera= round(n*(1-confianza))
    # VaR en porcentaje
    VaRPortP= sortport[fuera]
    # VaR en dinero
    VaRPortM= ExpTotal*VaRPortP
    return print("VaR en porciento:", VaRPortP, 
                 '\nVaR en dinero:', VaRPortM)

## ES con Títulos

In [3]:
def  ESPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                   start: 'Fecha inicio',
                   end: 'Fecha final',
                   titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                   confianza: 'Nivel de confianza del VaR (en decimal)'
                   ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Acomodarlo menor a mayor
    sortport= np.sort(Port)
    n= len(Port)
    # Nivel de confianza
    fuera= round(n*(1-confianza))
    # ES en porcentaje
    ESPortP= np.mean(sortport[0:fuera])
    # ES en dinero
    ESPortM= ESPortP*ExpTotal
    return print("ES en porciento:", ESPortP, 
                 '\nES en dinero:', ESPortM)

## Stress Test con Títulos

In [4]:
def  STPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                   start: 'Fecha inicio',
                   end: 'Fecha final',
                   titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)'
                   ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Rendimiento medio diario (media aritmética)
    ErP= np.mean(Port)
    # Volatilidad diaria (desviación estándar)
    sP= np.std(Port)
    # Stress en porcentaje
    STPortP= sP*5*-1
    # Stress en dinero
    STPortM= ExpTotal*STPortP
    return print("Stress Test en porciento:", STPortP, 
                 '\nStress Test en dinero:', STPortM)